In [11]:
# reference
# https://www.youtube.com/watch?v=OSA5fZZwEW4

from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import numpy as np
from os import listdir
import shutil
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import optim
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from os import listdir
import shutil
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import optim
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt


experiment_number = 7
isize = 256
iusize = 1024
num_epochs = 1000
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = 'max_split_size_mb:128'
img = Image.open("example_input_" + str(experiment_number) + ".png").convert("RGB")
transform = transforms.Compose([
    transforms.Resize(isize),
    transforms.CenterCrop(isize),
    transforms.ToTensor()
])
img = transform(img)
img = img.cuda()
avg_pool = nn.AvgPool2d(21, stride=1, padding=10)
avg_img = avg_pool(img)
img_var = avg_pool((img - avg_img).square()).sqrt()

In [12]:
class EllipseModel(nn.Module):
    def __init__(self, n, use_cuda=True):
        super(EllipseModel, self).__init__()
        self.num = n
        self.fr = 4  # Fourier terms
        self.use_cuda = use_cuda
        self.posx = torch.nn.Parameter(
            torch.rand((self.num), requires_grad=True))
        self.posy = torch.nn.Parameter(
            torch.rand((self.num), requires_grad=True))
        self.radius = torch.nn.Parameter(torch.randn((self.num), requires_grad=True))
        self.width = torch.nn.Parameter(torch.randn((self.num), requires_grad=True))
        self.length = torch.nn.Parameter(torch.randn((self.num), requires_grad=True))
        # self.fweights_sin = torch.nn.Parameter(
        #     torch.rand((self.fr, self.num), requires_grad=True))
        # self.fweights_cos = torch.nn.Parameter(
        #     torch.rand((self.fr, self.num), requires_grad=True))
        self.scale_x = torch.nn.Parameter(
            torch.rand((self.num), requires_grad=True))
        self.scale_y = torch.nn.Parameter(
            torch.rand((self.num), requires_grad=True))
        self.color = torch.nn.Parameter(
            torch.zeros((self.num, 3), requires_grad=True))
        self.alpha = torch.nn.Parameter(
            torch.zeros((self.num), requires_grad=True))
        self.rotate = torch.nn.Parameter(
            torch.rand((self.num), requires_grad=True))
        # self.order = torch.nn.Parameter(
        #     torch.rand((self.num), requires_grad=True))

    def forward(self, isize, t):
        posx =  self.posx
        posy =  self.posy
        radius = self.radius * 1.0 
        width = self.width.sigmoid() * float(1 << 12)
        length = self.length.sigmoid() * 128.0
        # colors = (self.colors).sigmoid()* 1.0 
        color = (self.color)
        alpha = (self.alpha).sigmoid() * 16.0
        # background = (self.background).sigmoid() + 0.5
        rotate = 3.141592 * self.rotate.sigmoid() * 8.0
        scale_x = (self.scale_x).sigmoid() * 16.0
        scale_y = (self.scale_y).sigmoid() * 16.0
        # order = self.order.sigmoid() * 8.0
        # fweights_sin = self.fweights_sin.sigmoid() * 2.0
        # fweights_cos = self.fweights_cos.sigmoid() * 2.0
        fr = self.fr  # Fourier terms

        X = torch.unsqueeze(torch.arange(
            0.5, isize + 0.5, 1) / isize, 0)
        Y = torch.unsqueeze(torch.arange(
            0.5, isize + 0.5, 1) / isize, 0)
        I = torch.unsqueeze(torch.ones(isize), 0)
        if self.use_cuda:
            X = X.cuda()
            Y = Y.cuda()
            I = I.cuda()

        X = X * I
        Y = torch.transpose(Y * I, 0, 1)
        X = X.unsqueeze_(0)
        Y = Y.unsqueeze_(0)

        X = X.expand(self.num, 1, isize, isize) - \
            posx.reshape(self.num, 1, 1, 1)
        Y = Y.expand(self.num, 1, isize, isize) - \
            posy.reshape(self.num, 1, 1, 1)
        X, Y = X * (rotate.reshape(self.num, 1, 1, 1).cos()) - Y * (rotate.reshape(self.num, 1, 1, 1,).sin()), \
            X * (rotate.reshape(self.num, 1, 1, 1).sin()) + \
            Y * (rotate.reshape(self.num, 1, 1, 1).cos())

        X = scale_x.reshape(self.num, 1, 1, 1) * X
        Y = scale_y.reshape(self.num, 1, 1, 1) * Y

        R = torch.sqrt(X.square() + Y.square())
        theta = -1.0 * (torch.acos(X / (R + 0.0001)) *
                        (1.0 * (Y > 0.0) - 1.0 * (Y < 0.0)))

        # fourier = torch.arange(1, fr + 1).expand(self.num,
        #                                          fr).permute(1, 0).reshape(fr, self.num, 1, 1, 1)

        # if self.use_cuda:
        #     fourier = fourier.cuda()

        # fourier = (fourier * (theta.expand(fr, self.num, 1, isize, isize)))
        # fourier = (fweights_sin.reshape(fr, self.num, 1, 1, 1) * fourier.sin() +
        #            fweights_cos.reshape(fr, self.num, 1, 1, 1) * fourier.cos()).sum(0)
        fourier = 0.0
        filter_values = (radius.reshape(self.num, 1, 1, 1) * (1.0 + 0.1 * fourier) - R)
        # sharpness = float(1 << 9)
        filters_soft = torch.exp(-width.reshape(self.num, 1, 1, 1) * filter_values * filter_values)
        filters2 = filters_soft * (filters_soft > 0.0)
        angle_alpha = torch.pow(1.0 - torch.cos(theta / 2.0), length.reshape(self.num, 1, 1, 1))
        return (color, alpha, angle_alpha, filters2, radius, theta)
num = 1024
model = EllipseModel(num, use_cuda=True)
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=0.005)
out = None
try:
    shutil.rmtree('.scratch')
except:
    pass
os.mkdir('.scratch')

def brightness(val):
   return torch.sqrt(0.241*(val[0]**2) + 0.691*(val[1]**2) + 0.068*(val[2]**2))

upsampler = nn.Upsample(scale_factor=2, mode='bicubic')

for epoch in range(0, num_epochs):
    with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        torch.cuda.empty_cache()

        color, alpha, angle_alpha, filters2, radius, theta = model(isize, -16.0)
        # rgb = torch.sum(filters2.expand(num, 3, isize, isize), (0),
                        # keepdim=True)/(torch.sum(filters2.expand(num, 3, isize, isize), (0), keepdim=True)+0.00001)
        # bg = torch.tensor([1.0]).expand(3, isize, isize).cuda() * torch.exp(-(torch.sum(torch.pow(1.0 - torch.cos(theta / 2.0), 8.0).reshape(num, 1, isize, isize) * alpha.reshape(num, 1, 1, 1).expand(num, 1, isize, isize) * filters2.expand(num, 1, isize, isize), (0))))
        radius_alpha = filters2
        # bg = torch.sum(angle_alpha.reshape(num, 1, isize, isize) * radius_alpha.reshape(num, 1, isize, isize) * color.reshape(num, 3, 1, 1).expand(num, 3, isize, isize) * torch.exp(-alpha).reshape(num, 1, 1, 1).expand(num, 1, isize, isize), (0)) / (torch.sum(angle_alpha.reshape(num, 1, isize, isize) * radius_alpha.reshape(num, 1, isize, isize) * torch.exp(-alpha).reshape(num, 1, 1, 1).expand(num, 1, isize, isize), (0)) + 0.00001)
        bg = torch.sum(angle_alpha.reshape(num, 1, isize, isize) * radius_alpha.reshape(num, 1, isize, isize) * color.reshape(num, 3, 1, 1).expand(num, 3, isize, isize) * torch.exp(-alpha).reshape(num, 1, 1, 1).expand(num, 1, isize, isize), (0)) / (torch.sum(angle_alpha.reshape(num, 1, isize, isize) * radius_alpha.reshape(num, 1, isize, isize) * torch.exp(-alpha).reshape(num, 1, 1, 1).expand(num, 1, isize, isize), (0)) + 0.00001)
        out = bg.cuda()
        # pic_loss = (((( (img-out) )))).abs().mean()
        # pic_loss = (((( (avg_img-out).abs() + (img-out).abs() ).square()))).mean().sqrt()
        pic_loss = (((( (img-out) ).square()))).mean().sqrt()
        # pic_loss = ((((brightness( (img-out)) ).square()))).mean().sqrt()
        # pic_loss = (brightness(img-out)/brightness(avg_img) ).square().mean().sqrt()
        # pic_loss = ((((( img_var*(img-out))/avg_img ).square()))).mean().sqrt()
        loss = pic_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # im = Image.fromarray((255.0 * out.clamp(0.0, 1.0).detach().permute(1,2,0).cpu().numpy()).astype(np.uint8), 'RGB').convert('RGB')
        im = out.clamp(0.0, 1.0).detach().permute(1,2,0).cpu().numpy()
        # im = im.resize((iusize, iusize), Image.LANCZOS)
        plt.imsave(".scratch/out_" + str(epoch) + ".png", im)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'model.ckpt')


Epoch [1/1000], Loss: 0.4353
Epoch [2/1000], Loss: 0.4351
Epoch [3/1000], Loss: 0.4348
Epoch [4/1000], Loss: 0.4344
Epoch [5/1000], Loss: 0.4340
Epoch [6/1000], Loss: 0.4336
Epoch [7/1000], Loss: 0.4332
Epoch [8/1000], Loss: 0.4328
Epoch [9/1000], Loss: 0.4323
Epoch [10/1000], Loss: 0.4318
Epoch [11/1000], Loss: 0.4313
Epoch [12/1000], Loss: 0.4308
Epoch [13/1000], Loss: 0.4303
Epoch [14/1000], Loss: 0.4297
Epoch [15/1000], Loss: 0.4291
Epoch [16/1000], Loss: 0.4285
Epoch [17/1000], Loss: 0.4279
Epoch [18/1000], Loss: 0.4273
Epoch [19/1000], Loss: 0.4267
Epoch [20/1000], Loss: 0.4260
Epoch [21/1000], Loss: 0.4253
Epoch [22/1000], Loss: 0.4246
Epoch [23/1000], Loss: 0.4238
Epoch [24/1000], Loss: 0.4230
Epoch [25/1000], Loss: 0.4222
Epoch [26/1000], Loss: 0.4214
Epoch [27/1000], Loss: 0.4205
Epoch [28/1000], Loss: 0.4196
Epoch [29/1000], Loss: 0.4187
Epoch [30/1000], Loss: 0.4177
Epoch [31/1000], Loss: 0.4168
Epoch [32/1000], Loss: 0.4157
Epoch [33/1000], Loss: 0.4146
Epoch [34/1000], Lo

In [2]:
class PointModel(nn.Module):
    def __init__(self, n, use_cuda=True):
        super(PointModel, self).__init__()
        self.num = n
        self.use_cuda = use_cuda
        self.posx = torch.nn.Parameter(
            torch.rand((self.num), requires_grad=True))
        self.posy = torch.nn.Parameter(
            torch.rand((self.num), requires_grad=True))

    def forward(self, isize):
        posx =  self.posx
        posy =  self.posy

        X = torch.unsqueeze(torch.arange(
            0.5, isize + 0.5, 1) / isize, 0)
        Y = torch.unsqueeze(torch.arange(
            0.5, isize + 0.5, 1) / isize, 0)
        I = torch.unsqueeze(torch.ones(isize), 0)
        if self.use_cuda:
            X = X.cuda()
            Y = Y.cuda()
            I = I.cuda()

        X = X * I
        Y = torch.transpose(Y * I, 0, 1)
        X = X.unsqueeze_(0)
        Y = Y.unsqueeze_(0)

        X = X.expand(self.num, 1, isize, isize) - \
            posx.reshape(self.num, 1, 1, 1)
        Y = Y.expand(self.num, 1, isize, isize) - \
            posy.reshape(self.num, 1, 1, 1)

        R = torch.sqrt(X.square() + Y.square())
        
        sharpness = float(1 << 9)
        ir = 0.5 / float(isize)

        filters = torch.exp(-R / ir)
        return (filters)

num = 1 << 12
model = PointModel(num, use_cuda=True)
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=0.005)
out = None
try:
    shutil.rmtree('.scratch')
except:
    pass
os.mkdir('.scratch')

def brightness(val):
   return torch.sqrt(0.241*(val[0]**2) + 0.691*(val[1]**2) + 0.068*(val[2]**2))

for epoch in range(0, num_epochs):
    with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        torch.cuda.empty_cache()

        filters = model(isize)
       
        bg = torch.tensor([1.0]).expand(3, isize, isize).cuda() * torch.exp(-4. * (torch.sum(filters.expand(num, 1, isize, isize), (0))))
        out = bg.cuda()
        pic_loss = (((( (img-out) ).square()))).mean().sqrt()
        loss = pic_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        im = out.clamp(0.0, 1.0).detach().permute(1,2,0).cpu().numpy()
        plt.imsave(".scratch/out_" + str(epoch) + ".png", im)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'model.ckpt')


Epoch [1/1000], Loss: 0.4229
Epoch [2/1000], Loss: 0.3966
Epoch [3/1000], Loss: 0.3821
Epoch [4/1000], Loss: 0.3734
Epoch [5/1000], Loss: 0.3673
Epoch [6/1000], Loss: 0.3627
Epoch [7/1000], Loss: 0.3589
Epoch [8/1000], Loss: 0.3555
Epoch [9/1000], Loss: 0.3524
Epoch [10/1000], Loss: 0.3498
Epoch [11/1000], Loss: 0.3477
Epoch [12/1000], Loss: 0.3455
Epoch [13/1000], Loss: 0.3435
Epoch [14/1000], Loss: 0.3419
Epoch [15/1000], Loss: 0.3405
Epoch [16/1000], Loss: 0.3391
Epoch [17/1000], Loss: 0.3379
Epoch [18/1000], Loss: 0.3367
Epoch [19/1000], Loss: 0.3357
Epoch [20/1000], Loss: 0.3348
Epoch [21/1000], Loss: 0.3339
Epoch [22/1000], Loss: 0.3331
Epoch [23/1000], Loss: 0.3323
Epoch [24/1000], Loss: 0.3316
Epoch [25/1000], Loss: 0.3310
Epoch [26/1000], Loss: 0.3304
Epoch [27/1000], Loss: 0.3298
Epoch [28/1000], Loss: 0.3293
Epoch [29/1000], Loss: 0.3288
Epoch [30/1000], Loss: 0.3283
Epoch [31/1000], Loss: 0.3280
Epoch [32/1000], Loss: 0.3276
Epoch [33/1000], Loss: 0.3271
Epoch [34/1000], Lo

In [ ]:
class PointModel(nn.Module):
    def __init__(self, n, use_cuda=True):
        super(PointModel, self).__init__()
        self.num = n
        self.use_cuda = use_cuda
        self.posx = torch.nn.Parameter(
            torch.rand((self.num), requires_grad=True))
        self.posy = torch.nn.Parameter(
            torch.rand((self.num), requires_grad=True))

    def forward(self, isize):
        posx =  self.posx
        posy =  self.posy

        X = torch.unsqueeze(torch.arange(
            0.5, isize + 0.5, 1) / isize, 0)
        Y = torch.unsqueeze(torch.arange(
            0.5, isize + 0.5, 1) / isize, 0)
        I = torch.unsqueeze(torch.ones(isize), 0)
        if self.use_cuda:
            X = X.cuda()
            Y = Y.cuda()
            I = I.cuda()

        X = X * I
        Y = torch.transpose(Y * I, 0, 1)
        X = X.unsqueeze_(0)
        Y = Y.unsqueeze_(0)

        X = X.expand(self.num, 1, isize, isize) - \
            posx.reshape(self.num, 1, 1, 1)
        Y = Y.expand(self.num, 1, isize, isize) - \
            posy.reshape(self.num, 1, 1, 1)

        R = torch.sqrt(X.square() + Y.square())
        
        sharpness = float(1 << 9)
        ir = 0.5 / float(isize)

        filters = torch.exp(-R / ir)
        return (filters)

num = 1 << 12
model = PointModel(num, use_cuda=True)
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=0.005)
out = None
try:
    shutil.rmtree('.scratch')
except:
    pass
os.mkdir('.scratch')

def brightness(val):
   return torch.sqrt(0.241*(val[0]**2) + 0.691*(val[1]**2) + 0.068*(val[2]**2))

for epoch in range(0, num_epochs):
    with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
        torch.cuda.empty_cache()

        filters = model(isize)
       
        bg = torch.tensor([1.0]).expand(3, isize, isize).cuda() * torch.exp(-4. * (torch.sum(filters.expand(num, 1, isize, isize), (0))))
        out = bg.cuda()
        pic_loss = (((( (img-out) ).square()))).mean().sqrt()
        loss = pic_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        im = out.clamp(0.0, 1.0).detach().permute(1,2,0).cpu().numpy()
        plt.imsave(".scratch/out_" + str(epoch) + ".png", im)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'model.ckpt')

In [13]:
plt.imsave("experiment_" + str(experiment_number) + ".png", im)

In [27]:
target_size = 256
from moviepy.editor import *
import cv2
fourcc = cv2.VideoWriter_fourcc(*"H264")
video = cv2.VideoWriter("experiment_video_" + str(experiment_number) + ".mp4", fourcc, 24, (target_size, target_size))
gen = {}
for n in range(0, num_epochs):
    try:
        image = cv2.imread(f".scratch/out_" + str(n) + ".png")

        h = image.shape[0]
        w = image.shape[1]
        
        # for y in range(0, h):
        #     for x in range(0, w):
        #         image[x][y][0] = 1.0 - image[x][y][0]

        image = cv2.resize(image, (target_size,target_size), interpolation=cv2.INTER_LANCZOS4)



        video.write(image)
        # print(f"Finished{n}")
    except:
        break
video.release()

In [4]:
torch.save(model.state_dict(), 'model.ckpt')

In [2]:

isize = 256
num = 1024
white = torch.tensor([1.0]).expand(num, 3, isize, isize)
model = EllipseModel(n=num, use_cuda=False)
model.load_state_dict(torch.load('model.ckpt'))
model = model.cpu()

filters_soft, filters2, radius, colors = model(isize, 0.001)
# mrgb = torch.sum(white*filters2*(filters_soft > 0.0), (2, 3),
#                     keepdim=True)/(torch.sum(filters2*(filters_soft > 0.0), (2, 3), keepdim=True)+0.00001)
rgb = colors.reshape((num, 3, 1, 1))*filters2.expand(num, 3, isize, isize)
out = torch.sum(filters2*rgb, 0)
out = out.clamp(0.0, 1.0)
plt.imsave("experiment.png", out.permute(1, 2, 0).detach().cpu().numpy())

In [121]:
from IPython.display import Video

Video("video.mp4")

class Model 